# Import Libraries

In [ ]:
import os
import cv2
import glob
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

sns.set()

# Data Exploration

In [ ]:
train_male = len(os.listdir("/kaggle/input/gender-classification-dataset/Training/male"))
train_female = len(os.listdir("/kaggle/input/gender-classification-dataset/Training/female"))
valid_male = len(os.listdir("/kaggle/input/gender-classification-dataset/Validation/male"))
valid_female = len(os.listdir("/kaggle/input/gender-classification-dataset/Validation/female"))
print(f"There are {train_male} male and {train_female} female in the training set")
print(f"There are {valid_male} male and {valid_female} female in the validation set")

In [ ]:
shapes = np.array([cv2.imread(path, -1).shape for path in np.random.choice(glob.glob("/kaggle/input/gender-classification-dataset/**/**/*"), 1000, replace=False)])
if not np.all(shapes==shapes[0]):
    print("Not all images have the same shape")
else:
    print(f"Almost all images are of shape {shapes[0]}")

In [ ]:
np.min(shapes, axis=0), np.max(shapes, axis=0), np.mean(shapes, axis=0), np.median(shapes, axis=0)

In [ ]:
median_shape = np.median(shapes, axis=0).astype(int)
min_shape = np.min(shapes, axis=0).astype(int)

In [ ]:
random_imgs = np.random.choice(glob.glob("/kaggle/input/gender-classification-dataset/**/**/*"), 30, replace=False)
plt.figure(figsize = (18, 5))
plt.suptitle(f"Example of {len(random_imgs)} random images from all the data")
for i in range(len(random_imgs)):
    plt.subplot(3,10,i+1)
    img = cv2.imread(random_imgs[i], -1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    plt.imshow(img, cmap = 'gray')
    plt.axis('off')

plt.show()

# Data Loader

We will create a class to load and generate data to our model. The class will be responsible of reading, shuffling and feeding the data into the fully connected network. 

In [ ]:
class DataLoader:
    def __init__(self, data_path, batch_size, shape, flatten):
        self._X, self._Y = self._read_data_path(data_path)
        self._batch_size = batch_size
        self.nb_iterations = self.__len__()//batch_size
        self._shape = shape
        self._flatten = flatten
    
    def __len__(self):
        return len(self._X)
    
    def get_shape(self):
        if self._flatten:
            return np.prod(self._shape)
        return self._shape + (3, )
    
    def _read_data_path(self, data_path):
        X = glob.glob(os.path.join(data_path, "male/*"))
        Y = [0]*len(X)
        
        X.extend(glob.glob(os.path.join(data_path, "female/*")))
        Y.extend([1]*(len(X)-len(Y)))
        
        return np.array(X), np.array(Y)
    

    def _read_single_image(self, img_path, label):
        img = tf.io.decode_png(
                tf.io.read_file(img_path), channels=3, dtype=tf.uint8
            )
        img = tf.image.resize(img, self._shape) / 255.
        if self._flatten:
            img = tf.image.rgb_to_grayscale(img)
            img = tf.reshape(img, (-1, ))
        return img, label
    
    def get_dataset(self):
        dataset = tf.data.Dataset.from_tensor_slices((self._X, self._Y))
        dataset = dataset.shuffle(
                buffer_size=self.__len__(), reshuffle_each_iteration=True
            ).repeat()
        dataset = dataset.map(self._read_single_image, tf.data.AUTOTUNE)
        return dataset.batch(batch_size=self._batch_size, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

# Blocks

In [ ]:
def conv_bn_relu(x, feature_maps, strides, kernel_size):
    x = tf.keras.layers.Conv2D(
          filters=feature_maps, kernel_size=kernel_size,
          strides=strides, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    return x


def residual_block(x, feature_maps_list, strides_list, kernel_size_list, drop_rate):
    residual = x

    if 2 in strides_list or x.shape[-1] < feature_maps_list[-1]:
        residual = conv_bn_relu(x, feature_maps_list[-1], strides=max(strides_list), kernel_size=1)

    if drop_rate > 0:
        x = tf.keras.layers.SpatialDropout2D(drop_rate)(x)

    for i in range(len(feature_maps_list)):
        x = conv_bn_relu(x, feature_maps_list[i], strides=strides_list[i], kernel_size=kernel_size_list[i])

    return tf.keras.layers.Add()([x, residual])


def dense_bn_relu(x, units, use_batch_norm, drop_rate):
    if drop_rate > 0:
        x = tf.keras.layers.Dropout(drop_rate)(x)
    x = tf.keras.layers.Dense(units=units)(x)
    if use_batch_norm:
        x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    return x

# Network

In [ ]:
def create_dense_model(input_size, hidden_units, use_batch_norm, drop_rate):
    input_layer = tf.keras.Input((input_size, ))
    x = input_layer
    for i, units in enumerate(hidden_units):
        rate = 0 if i == 0 else drop_rate
        x = dense_bn_relu(x, units, use_batch_norm=use_batch_norm, drop_rate=rate)
    
    return tf.keras.Model(input_layer, x)

def create_conv_model(input_size, config):
    input_layer = tf.keras.Input(input_size)
    x = input_layer
    for i in range(len(config["kernel_size"])):
        x = residual_block(x, config["feature_maps"][i], config["strides"][i],
                           config["kernel_size"][i], config["drop_rate"][i])
    
    x = tf.reduce_mean(x, axis=(1, 2))
    
    for units in config["dense"][:-1]:
        x = dense_bn_relu(x, units, use_batch_norm=True, drop_rate=0)
    
    x = tf.keras.layers.Dense(config["dense"][-1])(x)
        
    return tf.keras.Model(input_layer, x)

# Optimizer

In [ ]:
def create_optimizer(lr, nb_iterations, use_cosine_decay=False):
    if use_cosine_decay:
        alpha = 0.01
        cycle = 10
        decay_steps = cycle * nb_iterations
        lr = tf.keras.optimizers.schedules.CosineDecayRestarts(
                    lr,
                    decay_steps,
                    t_mul=1.0,
                    m_mul=1.0,
                    alpha=alpha,
                )
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    return optimizer

In [ ]:
batch_size=32
resize_shape=(100, 100)
training_path="/kaggle/input/gender-classification-dataset/Training"
validation_path="/kaggle/input/gender-classification-dataset/Validation"

training_data = DataLoader(training_path, batch_size, resize_shape, flatten=False)
validation_data = DataLoader(validation_path, batch_size, resize_shape, flatten=False)

In [ ]:
config = {
    "kernel_size": [[3], [3, 3], [3, 3], [3, 3]],
    "feature_maps": [[8], [16, 16], [32, 32], [64, 64]],
    "strides": [[1], [2, 1], [2, 1], [2, 1]],
    "drop_rate": [0, 0.2, 0.2, 0.2],
    "dense": [50, 1]
}
model = create_conv_model(training_data.get_shape(), config)
model.summary()

In [ ]:
learning_rate=0.01
optimizer = create_optimizer(learning_rate, training_data.nb_iterations, True)

In [ ]:
epochs=50
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

history = model.fit(training_data.get_dataset(),
                    validation_data=validation_data.get_dataset(),
                    epochs=epochs, steps_per_epoch=training_data.nb_iterations, validation_steps=validation_data.nb_iterations)

In [ ]:
train_loss = history.history['loss']
train_acc = history.history['accuracy']
valid_loss = history.history['val_loss']
valid_acc = history.history['val_accuracy']

In [ ]:
fig = plt.figure(figsize=(12, 6))
plt.suptitle("Learning Curves")
    
plt.subplot(121)
plt.title("cross entropy")
plt.plot(np.arange(1, len(train_loss)+1), train_loss, label='training', c='b')
plt.plot(np.arange(1, len(valid_loss)+1), valid_loss, label='validation', c='r')
plt.xlim(1, epochs + epochs//10)
plt.xticks(np.arange(0, epochs + epochs//10, epochs//10))
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), loc='upper right')

plt.subplot(122)
plt.title("accuracy")
plt.plot(np.arange(1, len(train_acc)+1), train_acc, label='training', c='b')
plt.plot(np.arange(1, len(valid_acc)+1), valid_acc, label='validation', c='r')
plt.xlim(1, epochs + epochs//10)
plt.xticks(np.arange(0, epochs + epochs//10, epochs//10))
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), loc='lower right')

# Test the model on new images

In [ ]:
fake_faces = []
for img_path in glob.glob("/kaggle/input/fake-faces/*"):
    img = cv2.cvtColor(cv2.imread(img_path, -1), cv2.COLOR_BGR2RGB)
    fake_faces.append(img)

In [ ]:
plt.figure(figsize=(12, 5))
for i in range(len(fake_faces)):
    plt.subplot(1, len(fake_faces), i+1)
    plt.imshow(fake_faces[i])
    plt.axis('off')

plt.show()

In [ ]:
batch_faces = []
for img in fake_faces:
    img = cv2.resize(img, resize_shape)
    img = img/255
    batch_faces.append(img)
    
batch_faces = np.array(batch_faces)

In [ ]:
preds = model(batch_faces)
preds = tf.math.sigmoid(preds)
genders =  ['male' if pred < 0.5 else "female" for pred in preds]
plt.figure(figsize=(18, 5))
for i in range(len(fake_faces)):
    plt.subplot(1, len(fake_faces), i+1)
    plt.title(f"gender: {genders[i]}\np(y=1/X,A)={np.round(preds[i][0], 2):.2f}", fontsize=10)
    plt.imshow(fake_faces[i])
    plt.axis('off')

plt.show()